#### Data Set Summary & Exploration

####1. Provide a basic summary of the data set. In the code, the analysis should be done using python, numpy and/or pandas methods rather than hardcoding results manually.


Number of training examples = 34799
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43
Number of Validation examples = 4410


####2. Include an exploratory visualization of the dataset.


Data Analysis
Here we training set classes are bit skewed, some of classes have more examples than others. This might lead to the neural model overfitted for classes with large number of examples and underfit for the classes with insufficient examples. To overcome this problem I thought of generating some data for the classes below threshold and bring their number to the mean.

Generating fake data
The idea is to increase the number of examples of images for certain classes particularly the ones with less number of examples (below mean is a good measure). Checking for the classes of the images by iterating through the images and generate that many number of examples for that class that helps reach the mean. The images are generated from existing images by using looping through to pick a random angle to rotate using scipy's rotate function.
These new set of images are again split between training and validation set to prevent overfitting of the model. train_test_split(X_train, y_train, stratify = y_train, test_size=0.1..) function is used to split the data. The newly generated training and validation data is added to the existing data set.



###Design and Test a Model Architecture

####1. Describe how you preprocessed the image data. What techniques were chosen and why did you choose these techniques? Consider including images showing the output of each preprocessing technique. Pre-processing refers to techniques such as converting to grayscale, normalization, etc. (OPTIONAL: As described in the "Stand Out Suggestions" part of the rubric, if you generated additional data for training, describe why you decided to generate additional data, how you generated the data, and provide example images of the additional data. Then describe the characteristics of the augmented training set like number of images in the set, number of images for each class, etc.)

PreProcessing
To preProcess the data is to prepare the data to be fed to the neural network or any other model. First thing in preprocessing was to :
Normalize: Any dataset would contain some noise with some very large values and some very small values. These datapoints lead to divergence of the model or might lead to convergence to some local minima. Normalizing the values help in avoiding these problems, keeping the values between .1 - .9.
RGB to GRAY: Data had 3 channels RGB, but looking at the examples and the problem statement, and from previous project I realized that this particular problem of classification does not actually require to provess 3 channels. So as to avoid complexity and to speedup the processing, I converted the data to Grayscale such that model is able to focus on important features of the images.
Reformat: Also we need to ensure that all the images follow the same size as expected by training model. This is to convert the data into row format such that it becomes easier to extract the features and model can be trained on those features. a. After grayscale it was required to convert it back into the array. b. Also, during the grayscale its shape got changed to 3232, I used numpy library to convert it back to 32321. Another important part was neural networks required the input to be of the shape 3232*1. Later for test data which had images of different sizes, I used numpy resize function to align its size according to the model.


####2. Describe what your final model architecture looks like including model type, layers, layer sizes, connectivity, etc.) Consider including a diagram and/or table describing the final model.

Model Architecture
The model is similar to LeNet model described in the lectures which includes couple of convolution layers with averaging pooling (avg_pool worked better than max_pool as in max_pool we loose some of the feature information as it takes max value of the pool but with avg_pool we do involve characteristics of all features, hence loose less information which leads to little better results). The pooling layer's function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control overfitting. The Pooling Layer operates independently on every depth slice of the input and resizes it spatially. The convolution layers are followed by full mesh topology with the addition of drop outs as the regulation technique to prevent overfitting. This improved the performance upto 3-5%.

Layer 1 - Convolutional. Input = 32x32x1. Output = 28x28x6 with relu for Activation. Avg-Pooling. Input = 28x28x6. Output = 14x14x6.
Layer 2 - Convolutional. Output = 10x10x16. with Relu for activation followed by pooling. AvgPooling - Input = 10x10x16. Output = 5x5x16.
We flatten the output and use the fully connected network beyond this layer.
Layer 3: Fully Connected. Input = 400. Output = 120. Then Relu and dropout with prob .7. The value of .7 is obtained using trial and error, this gives the best performance and the explaination would be it is the optimul balance between underfitting and overfitting.
Layer 4: Fully Connected. Input = 120. Output = 84 with relu for Activation and dropout with prob = 0.7
Layer 5: Fully Connected. Input = 84. Output = 43 this returns the logit.


####3. Describe how you trained your model. The discussion can include the type of optimizer, the batch size, number of epochs and any hyperparameters such as learning rate.

Training Parameters
Most of the parameters are same as of the LeNet architecture.
Learning Rate : .005 Increasing/Decreasing the rate decreases the accuracy of the model. Learning rate is used to help the model converge towards the minima; if we rate is too high, we might miss it and if it is too less it would be very slow and would require more processing and more time.
Reference: https://stackoverflow.com/questions/36162180/gradient-descent-vs-adagrad-vs-momentum-in-tensorflow
Optimizer: AdamOptimizer seems to work better than gradient descent. In comparison to SGD every parameter has different learning rate and model stores that information. It also stores the momentum changes for each of them separately. It fixes the momentum problem with SGD where when we are very close to the goal, our momentum in most of the cases is very high and it does not know that it should slow down. This can cause it to miss or oscillate around the minima.
softmax function
Epochs 40: With each epoch the model was learning better until it started memorizing. The accuracy kept improving until 40 epochs after which it decreases. 40 works best.
Batch_size: 128 This was derived from LeNet. I tried few batch sizes but 128 gave the best results.
Mean and Standard Deviation: 0 and .1 respectively. Model is generally designed to keep mean around 0 and equal variances.



####4. Describe the approach taken for finding a solution and getting the validation set accuracy to be at least 0.93. Include in the discussion the results on the training, validation and test sets and where in the code these were calculated. Your approach may have been an iterative process, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think the architecture is suitable for the current problem.

Solution Approach
With the base model taken from LeNet architecture, the model gives accuracy of 89%. So as to increase the accuracy of the model I tried to find the differences between this problem statement (classify traffic signs) with LeNet. One of the problem was that training data was not balanced which we fixed by generating more data for the corresponding classes. Another important thing was to add drop out to prevent overfitting of the model.
Next accuracy reached around 91.5% with these changes. So i figured optimizing for the hyperparameters would help and that indeed helped. I played around with epoch, keep probability of drop out, Learning Rate, Batch Size. And current configuration gave me the best results.



###Test a Model on New Images

####1. Choose five German traffic signs found on the web and provide them in the report. For each image, discuss what quality or qualities might be difficult to classify.

Acquiring New Images
Downloaded the german dataset and copied first 10 images for the test set but would be using only first 5. The description of images are as follows:
Keep right
turn right
Right-of-way at the next intersection
keep right
General Caution
Problem with these images is they have pretty less resolution. Moreover the sizes of each of the image were different. So as part of normalization and for network to process the image, I resized them to 32321 which again lead to some loss of the features. But since the test data chosen is pretty general in nature (e.g. speed sign with speed on it would be really difficult, as with each feature lost it would have lost some of the information) but our cases are quite general and model works decently on them.



####2. Discuss the model's predictions on these new traffic signs and compare the results to predicting on the test set. At a minimum, discuss what the predictions were, the accuracy on these new predictions, and compare the accuracy to the accuracy on the test set (OPTIONAL: Discuss the results in more detail as described in the "Stand Out Suggestions" part of the rubric).

Accuracy on Unknown Data
Model works decently well and gives 80% accuracy. To test the accuracy, I refered to CSV file and manually added labels for the test set. One of the reasons is the test set chosen is quite unique and based on the features it is easy to identify those signs. I would suspect that this model would not work that well on more complicated data e.g. speed sign to detect the speed. One of the improvements would be to get equal sized or generically crafted test set, since I modified it at my own will, there was definitely some loss in features and resolution which would cause model to degrade. Further increasing the epochs should also help.


####3. Describe how certain the model is when predicting on each of the five new images by looking at the softmax probabilities for each prediction. Provide the top 5 softmax probabilities for each image along with the sign type of each probability. (OPTIONAL: as described in the "Stand Out Suggestions" part of the rubric, visualizations can also be provided such as bar charts)

Top 5 Results for image: 0  are probabilities:  [ 52.40625      4.75464964   1.28858852  -7.93177462 -11.40607738]  Classes are:  [38 34 20  5 36]
Top 5 Results for image: 1  are probabilities:  [ 34.86618042  -0.74314737  -1.55930388  -3.95627379  -5.17855358]  Classes are:  [33  1 14  2 11]
Top 5 Results for image: 2  are probabilities:  [ 23.4937439    6.12285614   3.4959259    2.14273715  -0.94446498]  Classes are:  [11 30  6 21 27]
Top 5 Results for image: 3  are probabilities:  [ 43.74555969   3.59588289  -6.13170767  -6.13570499  -8.98134136]  Classes are:  [38 34 20  5 36]
Top 5 Results for image: 4  are probabilities:  [ 56.902771    13.25032997   2.87718248  -1.96136165  -4.79871082]  Classes are:  [18 26 12 25 38]
Top 5 classes and Probabilities
The above result shows the models top 5 results for each example and also shows the classes corresponding to those probabilities. The model could not identify image 4 which was keep right. It was bit wierd because of the case was identified by it. This is of course lower than the accuracy of 85% achieved on the test data, but still would be fairly good for not having curated the images exactly like whomever did so on the original dataset.
